<a href="https://colab.research.google.com/github/rodrigosarloaf/projeto_DL_23_1/blob/main/filtra_deter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pacotes
import pathlib
from osgeo import ogr, gdal, gdalconst
from skimage.morphology import disk, dilation, erosion
import os
import numpy as np
import shutil
from google.colab import drive
import cv2
from datetime import date, timedelta
import pandas as pd

In [ ]:
# Conectando ao drive
drive.mount('/content/drive', force_remount=True)
root_path = '/content/drive/My Drive/projeto_DL/'
os.chdir(root_path)

Mounted at /content/drive


In [ ]:
def converte_deter_tif(path_to_deter, output_path, patch_name, x_size, y_size, extent_tuple, d_begin, d_end):
  # Abrindo arquivo
  v_deter = ogr.Open(path_to_deter)
  l_deter = v_deter.GetLayer()
  # Definindo localização
  x_res = (extent_tuple[2]-extent_tuple[0])/x_size
  y_res = (extent_tuple[3]-extent_tuple[1])/y_size
  geo_transform = [extent_tuple[0], x_res, 0, extent_tuple[1], 0, y_res]
  # Nome de saída
  output = os.path.join(output_path, f'{patch_name}.tif')
  # Filtro de datas sendo aplicado
  where_ref = f'(cast(\"VIEW_DATE\" as character) <\'{d_end}\') AND ( cast(\"VIEW_DATE\" as character) >= \'{d_begin}\' )'
  # Objeto que manipula os dados
  target_ds = gdal.GetDriverByName('GTiff').Create(output, x_size, y_size, 1, gdal.GDT_Byte)
  target_ds.SetGeoTransform(geo_transform)
  l_deter.ResetReading()
  l_deter.SetAttributeFilter(where_ref)
  print (l_deter.GetFeatureCount())
  # Salvando
  gdal.RasterizeLayer(target_ds, [1], l_deter, burn_values=[1])
  # Limpando
  target_ds.FlushCache()
  del target_ds
  #
  return 1

In [ ]:
# Parâmetros fixos
output_path='/content/drive/My Drive/projeto_DL/deter_tratado/'
deterFile="/content/drive/My Drive/projeto_DL/dados_brutos/deter_public.shp"
x_size = 256*10
y_size = 256*10
extent= [-59.9206710380000018, -7.3125828444699836, -59.1668605229999969, -6.7566813625300162] 

In [ ]:
# Rodando em loop - versao mensal

# Construindo intervalos de datas
anos_rodar = ['2016','2017','2018','2019','2020','2021']
l_d_begin = []
for i in anos_rodar:
  for j in ['01','02','03','04','05','06','07','08','09','10','11','12']:
    l_d_begin.append(i+"/"+j+"/"+"01")

l_d_end = l_d_begin.copy()
del l_d_end[0]
l_d_end.append('2022/01/01')

# Loop
for i in range(len(l_d_begin)):
  patch_name=f"{i}"
  d_begin = l_d_begin[i]
  d_end = l_d_end[i]
  converte_deter_tif(deterFile, output_path + 'mensal/', patch_name, x_size, y_size, extent, d_begin, d_end)
  print((i+1)/len(l_d_begin))

0
0.013888888888888888
0
0.027777777777777776
0
0.041666666666666664
0
0.05555555555555555
0
0.06944444444444445
0
0.08333333333333333
0
0.09722222222222222
8300
0.1111111111111111
6007
0.125
7537
0.1388888888888889
2920
0.1527777777777778
67
0.16666666666666666
246
0.18055555555555555
325
0.19444444444444445
243
0.20833333333333334
427
0.2222222222222222
1238
0.2361111111111111
2417
0.25
2018
0.2638888888888889
1807
0.2777777777777778
6581
0.2916666666666667
4869
0.3055555555555556
3111
0.3194444444444444
2341
0.3333333333333333
1480
0.3472222222222222
1287
0.3611111111111111
2272
0.375
2438
0.3888888888888889
2611
0.4027777777777778
3784
0.4166666666666667
4737
0.4305555555555556
5140
0.4444444444444444
6334
0.4583333333333333
3555
0.4722222222222222
2057
0.4861111111111111
365
0.5
586
0.5138888888888888
672
0.5277777777777778
1494
0.5416666666666666
1605
0.5555555555555556
2398
0.5694444444444444
3245
0.5833333333333334
10357
0.5972222222222222
13078
0.6111111111111112
10992
0.625
3

In [ ]:
pd.DataFrame({'begin':l_d_begin, 'end':l_d_end, 'patch_name':np.arange(len(l_d_end))}).to_csv('/content/drive/My Drive/projeto_DL/deter_tratado/referencia_mes.csv')

In [ ]:
# Rodando em loop - versao semanal

# Pegando todos os domingos do ano
# https://stackoverflow.com/questions/2003870/how-can-i-select-all-of-the-sundays-for-a-year-using-python
def all_wdays(year, wday = 6):
   d = date(year, 1, 1)                    # January 1st
   d += timedelta(days = wday - d.weekday())  # 0 : Monday ... 6 : Sunday
   if d.year != year:
    d += timedelta(days = 7)
   while d.year == year:
      yield d
      d += timedelta(days = 7)

# Montando datas
# Começo da semana
anos_rodar = [2016,2017,2018,2019,2020,2021]
l_d_begin = []
for i in anos_rodar:
  for d in allsundays(i):
    l_d_begin.append(d)

# Fim da semana
l_d_end = []
for i in l_d_begin:
  l_d_end.append(i+timedelta(days = 6))

# Loop
for i in range(len(l_d_begin)):
  patch_name=f"{i}"
  d_begin = l_d_begin[i].strftime("%Y/%m/%d")
  d_end = l_d_end[i].strftime("%Y/%m/%d")
  converte_deter_tif(deterFile, output_path + 'semanal/', patch_name, x_size, y_size, extent, d_begin, d_end)
  print((i+1)/len(l_d_begin))

0
0.003194888178913738
0
0.006389776357827476
0
0.009584664536741214
0
0.012779552715654952
0
0.01597444089456869
0
0.019169329073482427
0
0.022364217252396165
0
0.025559105431309903
0
0.02875399361022364
0
0.03194888178913738
0
0.03514376996805112
0
0.038338658146964855
0
0.04153354632587859
0
0.04472843450479233
0
0.04792332268370607
0
0.051118210862619806
0
0.054313099041533544
0
0.05750798722044728
0
0.06070287539936102
0
0.06389776357827476
0
0.0670926517571885
0
0.07028753993610223
0
0.07348242811501597
0
0.07667731629392971
0
0.07987220447284345
0
0.08306709265175719
0
0.08626198083067092
0
0.08945686900958466
0
0.0926517571884984
0
0.09584664536741214
1804
0.09904153354632587
685
0.10223642172523961
2486
0.10543130990415335
703
0.10862619808306709
904
0.11182108626198083
2313
0.11501597444089456
210
0.1182108626198083
850
0.12140575079872204
2371
0.12460063897763578
2241
0.12779552715654952
1176
0.13099041533546327
2673
0.134185303514377
363
0.13738019169329074
1692
0.140575079

In [ ]:
pd.DataFrame({'begin':l_d_begin, 'end':l_d_end, 'patch_name':np.arange(len(l_d_end))}).to_csv('/content/drive/My Drive/projeto_DL/deter_tratado/referencia_semana.csv')